## Import Packages:
Please import the following packages to run the NCF model.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
## The path to the NCF model may be different for your case.
import models.ncf_model as ncf
import random

## Load and Preprocess Dataset:
For this notebook, I'll use the full interaction csv file you gave me. 

In [3]:
## Load in the dataset.
df = pd.read_csv('interactions.csv')

## Conver the number of counts to 1.
df['liked'] = (df['count'] > 0).astype(int)

In [4]:
## Create the numerical mappings for user ids.
user_mapping = {}
for index, user_id in enumerate(df['user'].unique()):
    user_mapping[user_id] = index

## Create the numerical mappings for song ids.
song_mapping = {}
for index, song_id in enumerate(df['track'].unique()):
    song_mapping[song_id] = index

## Map the raw user and song ids to the created numerical values.
df['user_id'] = df['user'].map(user_mapping)
df['song_id'] = df['track'].map(song_mapping)

In [ ]:
## Check the first five rows in the DataFrame.
df.head(5)

,user,track,count,liked,user_id,song_id
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,TRAUCNU128F42671EB,1,1,0,0
1,4bd88bfb25263a75bbdd467e74018f4ae570e5df,TRBFXMJ12903CB50F6,2,1,1,1
2,4bd88bfb25263a75bbdd467e74018f4ae570e5df,TRBBMHE128EF341D09,18,1,1,2
3,b64cdd1a0bd907e5e00b39e345194768e330d652,TRAUURC128E078EC6E,1,1,2,3
4,b64cdd1a0bd907e5e00b39e345194768e330d652,TRAQVTO128F14696A6,4,1,2,4


## Load in the NCF Model:
We now need to initialize the NCF model. These parameters were used to the train the model. 

In [7]:
## Determine the total number of unique users.
num_users = df['user_id'].nunique()
## Determine the total number of unique songs.
num_items = df['song_id'].nunique()
## Determine the number of latent features.
num_latent = 8

In [9]:
## Initalize the NCF model.
ncf_model = ncf.NCFModel(num_users, num_items, num_latent)

In [10]:
## Pass through random inputs to initialize the weights in the model.
ncf_model(np.array([np.array([0]), np.array([1])]))

## Now load in the trained weights from the MSD dataset.
ncf_model.load_weights('ncf_msd_weights.h5')

We can now try inputting random user and song interaction to get a predicted interaction score.

In [47]:
## Choose a random index to evaluate model on.
rand_index = random.choice(df.index)

## Extract the users and songs from this row.
user_test = np.array([df.iloc[rand_index]['user_id']])
song_test = np.array([df.iloc[rand_index]['song_id']])

## Create interaction array for both user and song.
user_song = [user_test, song_test]

## Input the interaction into the model to get a predicted interaction score.
pred_score = ncf_model.predict(user_song, verbose = 0).item()
print(f'Predicted Interaction: {pred_score}')

Predicted Interaction: 0.970494270324707


We can also do several interactions.

In [52]:
## Choose 10 random indices to evaluate model on.
rand_indices = random.sample(list(df.index), 10)

## Extract the users and songs from these rows.
user_tests = []
song_tests = []

for i in rand_indices:
    ## Get the users and items for specific row.
    user_temp = df.iloc[i]['user_id']
    song_temp = df.iloc[i]['song_id']
    
    ## Append the ids to their respective list.
    user_tests.append(user_temp)
    song_tests.append(song_temp)

## Convert the lists into numpy arrays.
user_tests = np.array(user_tests)
song_tests = np.array(song_tests)

## Create an interaction array for each user and song.
users_songs = [user_tests, song_tests]

## Input the interactions into the model and print out each predicted score.
pred_scores = ncf_model.predict(users_songs, verbose = 0)
print(pred_scores)

[[0.8547122 ]
 [0.74516106]
 [0.89123875]
 [0.65115595]
 [0.18540075]
 [0.03878032]
 [0.19459079]
 [0.0976558 ]
 [0.7819531 ]
 [0.3167414 ]]
